In [1]:
# 通过下⾯的LeNet代码，可以看出⽤深度学习框架实现此类模型⾮常简单。我们只需要实例化⼀
# 个Sequential块并将需要的层连接在⼀起。
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 定义模型
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),  # 经过卷积和池化后，得到的图像尺寸为5*5
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [3]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [4]:
net = net.cuda()
X = torch.rand(size=(256, 1, 28, 28), dtype=torch.float32)
X = X.cuda()
net(X)

tensor([[-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3090],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3090],
        ...,
        [-0.2566,  0.5279, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [5]:
X = torch.rand(size=(256, 1, 28, 28), dtype=torch.float32)
X = X.cuda()
net(X)

tensor([[-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3090],
        ...,
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3090],
        [-0.2566,  0.5279, -0.1335,  ..., -0.0890,  0.1135, -0.3091],
        [-0.2566,  0.5278, -0.1335,  ..., -0.0890,  0.1135, -0.3091]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [11]:
net.parameters().__next__()

Parameter containing:
tensor([[[[ 0.1799, -0.0721, -0.0436, -0.1411,  0.0047],
          [ 0.0844,  0.1236,  0.1950, -0.1987,  0.1201],
          [-0.1397, -0.0407, -0.1457,  0.0633,  0.0396],
          [ 0.1185,  0.0479, -0.1342,  0.1901, -0.0482],
          [-0.1370, -0.1256, -0.0106, -0.1640,  0.0157]]],


        [[[ 0.0678,  0.1633,  0.0160,  0.0469, -0.1323],
          [ 0.1537, -0.1858, -0.0068, -0.1857, -0.1141],
          [ 0.1646, -0.0645,  0.1549, -0.1694, -0.1794],
          [ 0.1377, -0.0564,  0.1541, -0.1935,  0.1794],
          [ 0.0578, -0.1256,  0.0261, -0.1752, -0.1388]]],


        [[[ 0.0247, -0.0630, -0.1767,  0.0076,  0.1652],
          [ 0.1117,  0.1789, -0.1199,  0.0240, -0.0068],
          [ 0.0110, -0.0967,  0.1965, -0.0888,  0.1382],
          [-0.1997, -0.1976,  0.1543, -0.1131,  0.0016],
          [-0.1933, -0.0788,  0.1845,  0.1193,  0.0987]]],


        [[[ 0.0461,  0.0232, -0.1689,  0.0140, -0.0931],
          [ 0.1944, -0.0520,  0.0792, -0.1790, -0.0783

In [21]:
# 现在我们已经实现了LeNet，让我们看看LeNet在Fashion-MNIST数据集上的表现。
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [22]:
for X, y in train_iter:
    print(X.shape)
    break

torch.Size([256, 1, 28, 28])


In [6]:
# d2l.evaluate_accuracy_gpu
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, nn.Module):
        net.eval() # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [7]:
# d2l.train_ch6
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec'
          f'on {str(device)}')

In [ ]:
lr, num_epochs = 0.9, 10
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

training on cuda:0


In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# 定义模型
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),  # 经过卷积和池化后，得到的图像尺寸为5*5
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

# 现在我们已经实现了LeNet，让我们看看LeNet在Fashion-MNIST数据集上的表现。
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)


lr, num_epochs = 0.9, 10
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

training on cuda:0


In [5]:
for X, y in train_iter:
    print(X.shape)
    break

torch.Size([256, 1, 28, 28])


In [1]:
import time

import torch
import torchvision.datasets
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# from model import *

# 准备数据集
train_data = torchvision.datasets.CIFAR10("./dataset_cifar10/train", train=True,
                                          transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("./dataset_cifar10/test", train=False,
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)

# 利用DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


# 创建网络模型
class Lyon(nn.Module):

    def __init__(self):
        super(Lyon, self).__init__()
        self.model1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x


lyon = Lyon()
if torch.cuda.is_available():
    lyon = lyon.cuda()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(lyon.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练次数
total_train_step = 0
# 记录训练次数
total_test_step = 0
# 训练轮数
epoch = 10

# 添加tensorboard
writer = SummaryWriter("./logs/train")

start_time = time.time()
for i in range(epoch):
    print("----- 第 {} 轮训练开始 -----".format(i + 1))
    # 训练步骤开始
    lyon.train()  # 可以不写
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        output = lyon(imgs)
        loss = loss_fn(output, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数：{}，Loss：{}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    lyon.eval()  # 评估步骤开始，可以不写
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = lyon(imgs)
            loss = loss_fn(outputs, targets)
            accuracy = (outputs.argmax(1) == targets).sum()  # outputs.argmax(1)将输出结果转换为targets的模式
            total_test_loss = total_test_loss + loss.item()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss：{}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(lyon, "./train_model/lyon_{}.pth".format(i))
    # 官方推荐模型保存方式
    # torch.save(lyon.state_dict(), "./lyon_{}.pth".format(i))
    print("模型已保存！")

writer.close()


31.6%


KeyboardInterrupt: 